In [2]:
#ライブラリインポート
from facenet_pytorch import MTCNN
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [3]:
#画像読み込み
def read_img(path):
    im=cv2.imread(path)
    img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return img_rgb
#画像出力
def im_show(im):
    plt.imshow(im)
    plt.show

In [4]:
#目と口の検出
def detect_mouth_eye(landmarks):
    mouth=[0,0]
    mouth[0]=(landmarks[0][3][0]+landmarks[0][4][0])/2
    mouth[1]=(landmarks[0][3][1]+landmarks[0][4][1])/2
    mouth_eye=[[0 for _ in range(2)]for _ in range(3)]
    mouth_eye[0]=list(landmarks[0][0])
    mouth_eye[1]=list(landmarks[0][1])
    mouth_eye[2]=mouth
    return mouth_eye
#ファイル書き込み
def write_file(path,mouth_eye):
    with open(path,mode='w') as f:
        for i in range(len(mouth_eye)):
                print(mouth_eye[i][0],mouth_eye[i][1],file=f)
#ファイル読み込み
def open_file(path):
    points=np.zeros((1,2))
    count=0
    with open(path) as f:
        for s_line in f:
            count +=1
            s=s_line.split()
            s_num=[int(float(i)) for i in s]
            s_num_array=np.array(s_num).reshape(1,2)
            if count==1:
                points=np.array(s_num_array)
            else:
                points=np.append(points,s_num_array,axis=0)
    return points

In [ ]:
digit=10000
zero_count=1
for i in range(4226,70000):
    #画像読み込み
    if digit<=i:
        digit*=10
        zero_count-=1
    img_path="images/"
    img_num="0"*zero_count
    img_num+=str(i)
    img_path+=img_num
    img_path+=".png"
    if i%1000==0:
        print(img_num)
    test=read_img(img_path)
    #MTCNN実行
    mtcnn=MTCNN(select_largest=False)
    #boxes:顔と検出した領域の四角形の座標情報, landmarks:目や鼻など顔のパーツごとの座標情報
    boxes, probs, landmarks = mtcnn.detect(test, landmarks=True)
    #landmarksをfileに書き込み
    path=f'landmarks/{img_num}_landmarks.txt'
    if landmarks is None:
        path=f'default.txt'
        with open(path,mode='w') as f:
            print(img_num,file=f)
        continue
    else:
        landmarks_reshape=np.array(landmarks.reshape(len(landmarks)*len(landmarks[0]),2))
        write_file(path,landmarks_reshape)
    #boxesをpathで指定したファイルに書き込み
    path=f'boxes/{img_num}_boxes.txt'
    with open(path,mode='w') as f:
        print(boxes[0][0],boxes[0][1],file=f)
        print(boxes[0][0],boxes[0][3],file=f)
        print(boxes[0][2],boxes[0][1],file=f)
        print(boxes[0][2],boxes[0][3],file=f)
    #口・目の情報を抜き取り
    mouth_eye=np.array(detect_mouth_eye(landmarks))
    #口・目の情報を保存するファイルパスを指定
    path=f'eye_mouth_points/{img_num}_points.txt'
    #指定したpathに口・目の情報を保存
    write_file(path,mouth_eye)

In [ ]:
#顔検出した結果を顔画像にプロットし，確認するためのコード

#対象の顔画像パス
test=read_img('images/34128.png')
#対象の画像のlandmarksを保存したファイルパス
points=open_file('landmarks/34128_landmarks.txt')
#対象の画像の目・口の情報のファイルパス
eye_mouth_points=open_file('eye_mouth_points/34128_points.txt')
#対象の画像のboxを保存したファイルパス
boxes=open_file('boxes/34128_boxes.txt')
fig, ax = plt.subplots(figsize=(16, 12))
ax.imshow(test)
ax.axis('off')
ax.scatter(points[:5, 0], points[:5, 1], s=100,marker="+")
ax.scatter(eye_mouth_points[:,0],eye_mouth_points[:,1],s=100,marker=".")
ax.scatter(boxes[:,0],boxes[:,1],s=100,marker="*")
fig.show()

In [ ]:
#エラー検出(MTCNNによって顔の認識が正しく行うことができあかったファイルリストの作成)
import os
digit=10
zero_count=4
for i in range(70000):
    if digit<=i:
        digit*=10
        zero_count-=1
    landmarks_path="landmarks/"
    img_num="0"*zero_count
    img_num+=str(i)
    landmarks_path+=img_num
    landmarks_path+="_landmarks.txt"
    if i%1000==0:
        print(img_num)
    if not os.path.isfile(landmarks_path):
        if not os.path.isfile('default.txt'):
            with open('default.txt',mode='w') as f:
                print(img_num,file=f)
        else:
            with open('default.txt',mode='a') as f:
                print(img_num,file=f)


In [ ]:
#手動検出(エラーによりMTCNNで顔の検出ができなかった画像に対する手動検出)
default_img=[]
with open('default.txt') as f:
    for s_line in f:
        default_img.append(s_line.replace('\n',''))
print(default_img)
i=11
path=f'Default_images/{default_img[i]}.png'
target_img=read_img(path)
fig, ax = plt.subplots(figsize=(16, 12))
ax.imshow(target_img)
#left_eye parameter
left_eye=[425,460]
#right_eye parameter
right_eye=[575,450]
#nose parameter
nose=[625,600]
#left_mouth　parameter
left_mouth=[475,750]
#right_mouth parameter
right_mouth=[600,725]
#center_mouth parameter
center_mouth=[560,740]
#landmarks parameter
mouth_eye=np.array([left_eye,right_eye,nose,left_mouth,right_mouth,center_mouth])
#plot images and landmarks
ax.scatter(mouth_eye[:,0],mouth_eye[:,1])
ax.grid(True)
ax.set_xticks(np.arange(0,1001,step=50))
ax.set_yticks(np.arange(0,1001,step=50))
fig.show()
mouth_eye=[left_eye,right_eye,nose,left_mouth,right_mouth,center_mouth]
path=f'Default_points/{default_img[i]}_points.txt'
write_file(path,mouth_eye)